In [ ]:
%run -n main.py

In [ ]:
lines = read_lines(DOTENV_PATH)
env = dict(parse_dotenv(lines))
openai.api_key = env['OPENAI_TOKEN']

# tasks

In [12]:
table = pd.read_csv('tasks/ru_user_oriented.csv')
table = table[~table.done.isnull()]
table = table.where(pd.notnull(table), None)
table = table[['id', 'instruction', 'input']]
ru_user_oriented_items = table.to_dict(orient='records')

# evals

## llama_7b_ru_turbo_alpaca_lora

In [1]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

model_name = 'IlyaGusev/llama_7b_ru_turbo_alpaca_lora'
device = 'cuda:0'

config = PeftConfig.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,

    # LLM.int8() requires Turing or Ampere GPUs.
    # WARNING: No libcudart.so found! Install CUDA or the cudatoolkit package (anaconda)!
    # WARNING: Compute capability < 7.5 detected! Only slow 8-bit matmul is supported for your GPU!
    # load_in_8bit=True,
    
    # A device map needs to be passed to run convert models into mixed-int8
    # device_map='auto'
    
    torch_dtype=torch.float16    
).to(device)


Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

In [2]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [7]:
model = PeftModel.from_pretrained(model, model_name, anc=torch.float16)

In [10]:
%run -n main.py
lines = read_lines('evals/gusev_7b_ru_alpaca_lora.jsonl')
eval_items = list(parse_jsonl(lines))
cache_ids = {_['id'] for _ in eval_items}

In [13]:
items = [
    _ for _ in ru_user_oriented_items
    if _['id'] not in cache_ids
]
for item in log_progress(items):
    prompt = user_oriented_ru_alcapa_prompt(item)
    output = ru_alpaca_complete(prompt, model, tokenizer)
    item = {
        'id': item['id'],
        'prompt': prompt,
        'output': output
    }
    eval_items.append(item)

  0%|                                                                                                                                                                                | 0/55 [00:00<?, ?it/s]/home/yc-user/.env/lib/python3.10/site-packages/transformers/generation/utils.py:1219: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
  0%|                                                                                                                                                                                | 0/55 [00:00<?, ?it/s]


RuntimeError: expected scalar type Half but found Float

## gpt-3.5-turbo

In [ ]:
lines = read_lines('evals/openai_turbo.jsonl')
eval_items = list(parse_jsonl(lines))
cache_ids = {_['id'] for _ in eval_items}

In [ ]:
items = [
    _ for _ in ru_user_oriented_items
    if _['id'] not in cache_ids
]
for item in log_progress(items):
    prompt = user_oriented_openai_prompt(item)
    output = openai_chat_complete(prompt, model='gpt-3.5-turbo')
    item = {
        'id': item['id'],
        'prompt': prompt,
        'output': output
    }
    eval_items.append(item)

In [ ]:
%run -n main.py
lines = format_jsonl(eval_items)
write_lines('evals/openai_turbo.jsonl', lines)

## text-davinci-003

In [ ]:
lines = read_lines('evals/openai_davinci_003.jsonl')
eval_items = list(parse_jsonl(lines))
cache_ids = {_['id'] for _ in eval_items}

In [ ]:
items = [
    _ for _ in ru_user_oriented_items
    if _['id'] not in cache_ids
]
for item in log_progress(items):
    prompt = user_oriented_openai_prompt(item)
    output = openai_complete(prompt, model='text-davinci-003')
    item = {
        'id': item['id'],
        'prompt': prompt,
        'output': output
    }
    eval_items.append(item)

In [ ]:
%run -n main.py
lines = format_jsonl(eval_items)
write_lines('evals/openai_davinci_003.jsonl', lines)

## text-davinci-002

In [ ]:
lines = read_lines('evals/openai_davinci_002.jsonl')
eval_items = list(parse_jsonl(lines))
cache_ids = {_['id'] for _ in eval_items}

In [ ]:
items = [
    _ for _ in ru_user_oriented_items
    if _['id'] not in cache_ids
]
for item in log_progress(items):
    prompt = user_oriented_openai_prompt(item)
    output = openai_complete(prompt, model='text-davinci-002')
    item = {
        'id': item['id'],
        'prompt': prompt,
        'output': output
    }
    eval_items.append(item)

In [ ]:
%run -n main.py
lines = format_jsonl(eval_items)
write_lines('evals/openai_davinci_002.jsonl', lines)

## davinci-001

In [ ]:
lines = read_lines('evals/openai_davinci_001.jsonl')
eval_items = list(parse_jsonl(lines))
cache_ids = {_['id'] for _ in eval_items}

In [ ]:
items = [
    _ for _ in ru_user_oriented_items
    if _['id'] not in cache_ids
]
for item in log_progress(items):
    prompt = user_oriented_openai_prompt(item)
    output = openai_complete(prompt, model='davinci')
    item = {
        'id': item['id'],
        'prompt': prompt,
        'output': output
    }
    eval_items.append(item)

In [ ]:
%run -n main.py
lines = format_jsonl(eval_items)
write_lines('evals/openai_davinci_001.jsonl', lines)